In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
!nvidia-smi

In [ ]:
# ****************************************************************
# ****************************************************************
# ****************************************************************

pOoiInitial = 'laptop'
pOoi   = 'laptop'
pOoiPr = '69'
cOoi   = 'waterBottle'
cOoiPr = '23'

# ****************************************************************
# ****************************************************************
# ****************************************************************
# 
# 
# # File Two: Read the sorts, build the events and store in a .txt file.


import fileinput
import sys

def initialFluent(file, searchExp, replaceExp):
    #https://stackoverflow.com/questions/39086/search-and-replace-a-line-in-a-file-in-python
    for line in fileinput.input(file, inplace=1):
        if searchExp in line:
            line = line.replace(searchExp,replaceExp)
        sys.stdout.write(line)

def timepoint():
    happensCount = open('./lp/events.lp')
    lines = happensCount.readlines()
    print(lines[-1])
    start = '), '
    end = ').'
    extracted = lines[-1][lines[-1].find(start)+len(start):lines[-1].rfind(end)]
    # time = int((str(lines).count('happens(')))
    time = str(int(extracted)+1)
    return time

def event(pOoi, cOoi, time):
    pEChanOOI = 'happens(ooiChange(' + pOoi + ',' + ' ' + cOoi + '), ' + time + ').'
    print (pEChanOOI)
    with open('./lp/events.lp', 'a') as out:
        out.write('\n{}'.format(pEChanOOI))
        print ('./lp/events.lp is stored')

In [ ]:
# Test cell

file = './lp/events.lp'
searchExp  = 'holdsAt(oOI(waterBottle'
replaceExp = 'holdsAt(oOI(' + pOoiInitial

initialFluent(file, searchExp, replaceExp)

pOoi = 'chair'
cOoi = 'sugar'

# happens(ooiChange(laptop, waterBottle), 0).
# happens(ooiChange(waterBottle, spoon), 4).
# happens(ooiChange(spoon, cup), 5).
# happens(ooiChange(cup, sugar), 6).
# happens(ooiChange(sugar, teapot), 7).

time = timepoint()
event(pOoi, cOoi, time)

In [ ]:
# Run the reasoning system.
import os
import subprocess

time = timepoint()

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'

clingoTime = int(time) + 2
clingoTime = str(clingoTime)

!chmod +x ./clingo/clingoLinux

"""
command = "clingo\\clingoWindows.exe -c maxtime=3 -n 1 " + \
            "lp\\decOrigin2.lp " + \
            "lp\\sorts.lp " + \
            "lp\\events.lp " + \
            "lp\\activities.lp " + \
            "lp\\ecasp888.lp " + \
            "lp\\ecasp8882.lp " + \
            "lp\\ecasp88888.lp | " + \
            "clingo\\format-output.exe 3 > " + \
            "inference\\ecasp88.txt"""
            
command = mainPath + 'clingo/clingoLinux -c maxtime=' + clingoTime + ' -n 1 ' + \
            mainPath + 'lp/decOrigin2.lp ' + \
            mainPath + 'lp/sorts.lp ' + \
            mainPath + 'lp/events.lp ' + \
            mainPath + 'lp/activities.lp ' + \
            mainPath + 'lp/ecasp888.lp ' + \
            mainPath + 'lp/ecasp8882.lp ' + \
            mainPath + 'lp/ecasp88888.lp | ' + \
            mainPath + 'clingo/format-output13042022_4.bin ' + clingoTime + ' > ' + \
            mainPath + 'inference/ecasp88.txt'

print(command)
osCommand = os.system(command)

In [ ]:
# Read the atomic activities and their respective timepoints
## Read answer set extracts by timepoints

import re

def answerSet(pathToAnswerSet):
    text_file = open(pathToAnswerSet, "r")
    data = text_file.read()
    text_file.close()

    regex = r"((?:[^\n][\n]?)+)"

    matches = re.finditer(regex, data)

    grouped = []

    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            grouped.append(match.group(groupNum))
    return grouped

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'        
file = "inference/ecasp88.txt"

grouped = answerSet(mainPath + file)

## Read the atomic activities from the answer set extracts

def findAtomicActivities(fromGrouped):
    start = 'Activity('
    end = ')'
    extracted = fromGrouped[fromGrouped.find(start)+len(start):fromGrouped.rfind(end)]
    previous, current, probability = extracted.split(',')
    return previous, current, probability

timepoints = []
activity   = []
object     = []
probabilit = []

match = "+atomicActivity("

for ff, gg in enumerate(grouped):
    m = re.search('\+atomicActivity\((.+?),.+\)', grouped[ff])
    if m:
        found = m.group(0)
        previous, current, probability = findAtomicActivities(found)
        timepoints.append(grouped[ff][0])
        activity.append(previous)
        object.append(current)
        probabilit.append(probability)

print(timepoints)
print(activity)
print(object)
print(probabilit)

In [ ]:
# Read the composite activity predicate and extract the atomic activities

def atomicActOntology(ontology):
    start = 'Activity('
    end = ')'
    extracted = ontology[ontology.find(start)+len(start):ontology.rfind(end)]
    atomic = extracted.split(', ')
    return atomic

fileOntology = "lp/ecasp88888.lp"
ontology = answerSet(mainPath + fileOntology)

numberOfAtomic = ontology[3].count(',')
atomic = atomicActOntology(ontology[3])
print(atomic)

TT = []

# for hh in range(numberOfAtomic):
#     # print(atomic[hh])
#     # print(activity[0])
#     # if atomic[hh] in activity:
#     TT.append(timepoints[activity.index(atomic[hh])])
#     print(TT)

res = []
i = 0
while (i < len(activity)):
    if (atomic.count(atomic[i]) > 0):
        res.append(i)
    i += 1
print(res)
  
# # printing result 
# print("The matching element Indices list : " + str(res))

In [ ]:
# Read file ecasp8882.lp

T = TT[3]
T1 = TT[2]
T2 = TT[1]
T3 = TT[0]

composite = "happens(compositeChange(A1, A2, A3, A4), " + str(T) + ") :- \n \
				holdsAt(atomicActivity(A4, O4, Pa4), " + str(T) + "), \n \
				holdsAt(atomicActivity(A3, O3, Pa3), " + str(T1) + "), \n \
				holdsAt(atomicActivity(A2, O2, Pa2), " + str(T2) + "), \n \
				holdsAt(atomicActivity(A1, O1, Pa1), " + str(T3) + "), \n \
				contextCompositeActivity(A1, A2, A3, A4, A5), \n \
				objectOI(O1, Po1), objectOI(O2, Po2), \n \
				objectOI(O3, Po3), objectOI(O4, Po4),\n \
				affordance(A1), affordance(A2), affordance(A3),\n \
				affordance(A4), affordance(A5),	time(T)."

with open('./lp/ecasp8882.lp', 'w') as out:
	out.write(composite)
	print ('./lp/ecasp8882.lp is stored')

In [ ]:
# Problem in matching the elements lists of the Answer Set and those of the ontology
# to perform the timepoints extractions to add them in the previous cell for inferring
# the composite activity.

In [ ]:
import re

atomicActivity = open(mainPath + "inference/ecasp88.txt")
line = atomicActivity.readlines()

print(line)

for i in range(len(line)):
    # atomicActivity = re.search('\+atomicActivity\((.+?),', line[i])
    atomicActivity = re.search('((?:[^\n][\n]?)+)', line[i])
    print(atomicActivity, i)

In [ ]:
import re

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'
atomicActivity = open(mainPath + "inference/ecasp88.txt")
line = atomicActivity.readlines()

#open text file in read mode
text_file = open(mainPath + "inference/ecasp88.txt", "r")
 
#read whole file to a string
data = text_file.read()
 
#close file
text_file.close()
 
print(data)

line = str(line)
# line = line.replace("',","")
# line = line.replace("'","")
print(line)

regex = r"((?:[^\n][\n]?)+)"

test_str = ("\n"
	"==========\n"
	"Answer: 1\n\n"
	"0\n"
	"happens(atomicChange(waterBottle,office),0)\n"
	"happens(ooiChange(laptop,waterBottle),0)\n"
	"oOI(laptop,70)\n"
	"currentLocation(office,56)\n\n"
	"1\n"
	"-oOI(laptop,70)\n"
	"+atomicActivity(serve,waterBottle,49)\n"
	"currentLocation(office,56)\n"
	"happens(atomicChange(spoon,office),1)\n"
	"happens(ooiChange(waterBottle,spoon),1)")

matches = re.finditer(regex, data)

grouped = []

for matchNum, match in enumerate(matches, start=1):
    # print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        grouped.append(match.group(groupNum))
        # print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

In [ ]:
import re

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'
text_file = open(mainPath + "inference/ecasp88.txt", "r")
data = text_file.read()
text_file.close()

regex = r"((?:[^\n][\n]?)+)"

matches = re.finditer(regex, data)

grouped = []

for matchNum, match in enumerate(matches, start=1):
    # print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        grouped.append(match.group(groupNum))
        # print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

In [ ]:
print(grouped[5])

In [ ]:
import re

mainPath = '/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/'
atomicActivity = open(mainPath + "inference/ecasp88.txt")
line = atomicActivity.readlines()

#line = str(line)
# print(line)

spaces = []

for s in range(len(line)):
	if line[s] == '\n':
		spaces.append(s)

print(spaces)

for ss in range(len(spaces)):
	# print(ss)
	# print(type(ss))
    # print(line[ss])
    # print(line[ss+1])
	for rr in range(spaces[ss],spaces[ss+1],1):
		print(line[rr])

In [ ]:
# File One: Read the sorts and store in a .txt file.


# ****************************************************************
# ****************************************************************
# ****************************************************************

pLoc   = 'office'
pLocPr = '78'
cLoc   = 'desktop'
cLocPr = '65'

pOoi   = 'cup'
pOoiPr = '69'
cOoi   = 'chair'
cOoiPr = '23'

# ****************************************************************
# ****************************************************************
# ****************************************************************

pLocation = 'location(' + pLoc + ',' + ' ' + pLocPr + ').'
cLocation = 'location(' + cLoc + ',' + ' ' + cLocPr + ').'

pObjectOI = 'objectOI(' + pOoi + ',' + ' ' + pOoiPr + ').'
cObjectOI = 'objectOI(' + cOoi + ',' + ' ' + cOoiPr + ').'

print (pLocation)
print (cLocation)
print (pObjectOI)
print (cObjectOI)

with open('./lp/sorts.lp', 'w') as out:
    out.write('{}\n{}\n{}\n{}\n'.format(pLocation, cLocation, pObjectOI, cObjectOI))
    print ('sorts.lp is stored')

# File Two: Read the sorts, build the events and store in a .txt file.


pEChanLoc = 'happens(changeOOI(' + pOoi + ',' + ' ' + cOoi + '), 1).'

pEChanOOI = 'happens(changeLocation(' + pLoc + ',' + ' ' + cLoc + '), 1).'

print (pEChanLoc)
print (pEChanOOI)

csloi = "not releasedAt(F,0) :- fluent(F)."

with open('./lp/events.lp', 'w') as out:
    out.write('{}\n{}\n{}\n'.format(csloi, pEChanLoc, pEChanOOI))
    print ('./lp/events.lp is stored')

In [ ]:
# Run the reasoning system.
import os

command = "./clingo/clingoWindows.exe -c maxtime=3 -n 1 ./lp/decOrigin2.lp ./lp/sorts.lp ./lp/events.lp ./lp/activities.lp ./lp/ecasp888.lp ./lp/ecasp88888.lp | ./clingo/format-output.exe 3 >./inference/ecasp88.txt"
# command = "./clingo/clingoLinux -c maxtime=3 -n 1 ./lp/decOrigin2.lp ./lp/sorts.lp ./lp/events.lp ./lp/activities.lp ./lp/ecasp888.lp ./lp/ecasp88888.lp | ./clingo/format-output 3 >./inference/ecasp88.txt"

os.system(command)

In [ ]:
# Read the intention.

intention = open("./inference/ecasp88.txt")
line = intention.readlines()
print (line[15])

match = "+intention("

if match in line[15]:
    print ("Yes, it is. Hihi!!")

In [ ]:
# Extract the intention.

import re

m = re.search('\+intention\((.+?),', line[15])
if m:
    found = m.group(1)

In [ ]:
# Resize video

import cv2
import numpy as np

pathToResize = mainPath + "input/"

print(pathToResize + "Egocentric18022022_3C_2_CASE2022_detected.mp4")

# cap = cv2.VideoCapture(pathToResize + "Robot17022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_2C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_2C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_3C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_3C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric17022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_2C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_2C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_2_detected.mp4")
cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_2_CASE2022_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_1FPS_detected.mp4")

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(pathToResize + "Robot17022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_2C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_2C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_3C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_3C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_2C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_2C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_2_detectedResized.mp4",fourcc, 3.0, (1920,1080))
out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_2_CASE2022_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
 
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(1920,1080))
        out.write(b)
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# Reduce FPS

import cv2

# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Robot17022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Robot18022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Robot18022022_2C.mp4"
pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/YDXJ0133.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/kitchen2.mp4"

# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Egocentric17022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Egocentric18022022C.mp4"
# pathToReduce = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/input/Egocentric18022022_2C.mp4"

# pathOutputFrames = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputEgocentric/frame"
pathOutputFrames = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/output/frames/intentionICTAI020072022"

vidcap = cv2.VideoCapture(pathToReduce)

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000) 
    hasFrames,image = vidcap.read() 
    if hasFrames:
        cv2.imwrite(pathOutputFrames + str(sec) + ".jpg", image)
    return hasFrames 

sec = 0
frameRate = 1 #it will capture image in each 0.5 second
success = getFrame(sec)
while success:
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
# Correct for inserting Gaze Data

import json
from tqdm import tqdm
import cv2
import json
import numpy as np
from opencv_jupyter_ui import cv2_imshow

radius = 20
color = (255, 255, 255)
thickness = 2

# vid = "/raid/home/labuseraber/environment/testEnvironment/yolov5/runs/detect/exp7/kitchen4.mp4"
vid = "/raid/home/labuseraber/environment/testEnvironment/yolov5/runs/detect/exp/kitchen2.mp4"
vidcap = cv2.VideoCapture(vid)
success, img = vidcap.read()
height, width, channels = img.shape
count = 0

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('./kitchen2Gaze23072022.mp4', fourcc, 25, (width, height))

def read(lineNumber):
    # r = open('/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/gaze.json')
    r = open('/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/gazedata')
    line = r.readlines()
    e = json.loads(line[lineNumber])
    # if e["data"]:
    #     x = e["data"]["gaze2d"][0]
    #     y = e["data"]["gaze2d"][1]
    #     img = cv2.circle(img, (int(x*width), int(y*height)), radius, color, thickness)
    #     video.write(img)
    # else:
    #     video.write(img)
    # return x, y
    if e["data"]:
        x = e["data"]["gaze2d"][0]
        y = e["data"]["gaze2d"][1]
        return x, y
    else:
        x=0
        y=0
        return x, y

for i in tqdm(range(5291)):
    success, img = vidcap.read()
    count = count + 4
    # print(i, count)
    x, y = read(count)
    img = cv2.circle(img, (int(x*width), int(y*height)), radius, color, thickness)
    # cv2_imshow('test',img)
    video.write(img)

cv2.destroyAllWindows()
video.release()

# Starting of the reasoning in stream

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
!nvidia-smi

import cv2
count = cv2.cuda.getCudaEnabledDeviceCount()
print(count)

In [ ]:
import cv2
import time
import sys
import numpy as np
from tqdm import tqdm
import os

mainPath = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/"

# pathToVideo  = mainPath + "input/Robot_1FPS.mp4"
# pathToVideo  = mainPath + "input/Robot17022022C_1FPS.mp4"
# pathToVideo  = mainPath + "input/Robot18022022C.mp4"
# pathToVideo  = mainPath + "input/Robot18022022C_1FPS.mp4"
# pathToVideo  = mainPath + "input/Robot18022022_2C.mp4"
# pathToVideo  = mainPath + "input/Robot18022022_2C_1FPS.mp4"
# pathToVideo  = mainPath + "input/Robot18022022_3C.mp4"
# pathToVideo  = mainPath + "input/Robot18022022_3C_1FPS.mp4"

# pathToVideo  = mainPath + "input/Egocentric_1FPS.mp4"
# pathToVideo  = mainPath + "input/Egocentric17022022C_1FPS.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022C.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022C_1FPS.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022_2C.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022_2C_1FPS.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022_3C.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022_3C_2.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022_3C_2_CASE2022.mp4"
# pathToVideo  = mainPath + "input/Egocentric18022022_3C_1FPS.mp4"
pathToVideo  = mainPath + "input/YDXJ0133_1FPS.mp4"
pathToVideo  = mainPath + "input/kitchen2_1FPS.mp4"
pathToVideo  = mainPath + "input/kitchen2.mp4"
pathToVideo  = mainPath + "input/YDXJ0133.mp4"

# pathToONNX   = mainPath + "modelsONNX/best4.onnx"                                 # Robot Occluded objects

# pathToONNX   = mainPath + "modelsONNX/egocentric04022022.onnx"
# pathToONNX   = mainPath + "modelsONNX/egocentricCoco17022022.onnx"
# pathToONNX   = mainPath + "modelsONNX/egocentricCoco18022022_2.onnx"
pathToONNX   = mainPath + "modelsONNXs/intentionICTAI020072022.onnx"
pathToONNX = "/raid/home/labuseraber/environment/testEnvironment/yolov5/runs/train/exp28/weights/intentionICTAI06072022.onnx"

# pathToONNX   = mainPath + "modelsONNX/robot04022022.onnx"
# pathToONNX   = mainPath + "modelsONNX/robotCoco17022022.onnx"
# pathToONNX   = mainPath + "modelsONNX/robotCoco18022022_2.onnx"
# pathToONNX   = mainPath + "modelsONNX/robotCoco19022022.onnx"

pathToLabels = mainPath + "labels"

def build_model(is_cuda):
    net = cv2.dnn.readNet(pathToONNX)
    if is_cuda:
        print("Attempty to use CUDA")
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
    else:
        print("Running on CPU")
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
    return net

INPUT_WIDTH = 640
INPUT_HEIGHT = 640
# INPUT_WIDTH = 1920
# INPUT_HEIGHT = 1080

def detect(image, net):
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (640, 640), swapRB=True, crop=False)
    # blob = cv2.dnn.blobFromImage(image, 1/255.0, (640, 640), swapRB=True, crop=False)
    net.setInput(blob)
    preds = net.forward()
    return preds

def load_capture():
    capture = cv2.VideoCapture(pathToVideo)
    return capture

def format_yolov5(frame):
    row, col, _ = frame.shape
    _max = max(col, row)
    result = np.zeros((_max, _max, 3), np.uint8)
    result[0:row, 0:col] = frame
    return result

labelsPath = os.path.sep.join([pathToLabels, "intentionICTAI09072022.txt"]) # "labelsRobot.txt"]) # "labelsEgocentric.txt"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

# is_cuda = len(sys.argv) > 1 and sys.argv[1] == "cuda"
is_cuda = "cuda"
nett = build_model(is_cuda)

def YOLO_OBJECTS(image, frame_w, frame_h, layerOutputs):
    boxes = []
    confidences = []
    classIDs = []
    labels = []
    centPoints = []
    ignoredClasses = ['phone','person','diningtable','frisbee','knife','toilet','snowboard','bed','tie','refrigerator','bottle','donut']
    
    output_data = layerOutputs[0]
    
    rows = output_data.shape[0]
    
    image_width, image_height, _ = image.shape

    x_factor = image_width  / INPUT_WIDTH  #frame_w   # INPUT_WIDTH
    y_factor = image_height / INPUT_WIDTH  #frame_h   # INPUT_HEIGHT
    
    for r in range(rows):
        row = output_data[r]
        confidence = row[4]
        if confidence >= 0.5:
            scores = row[5:]
            _, _, _, max_indx = cv2.minMaxLoc(scores)
            classID = max_indx[1]
            if (scores[classID] > .25):
                confidences.append(confidence)
                classIDs.append(classID)

                x, y, w, h = int(row[0].item()), int(row[1].item()), int(row[2].item()), int(row[3].item())
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    conf = []
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.25)
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # draw a bounding box rectangle and label on the frame
            if  LABELS[classIDs[i]] not in ignoredClasses :
                centPoints.append((int(x+(w/2)) ,int(y+(h/2))))
                # color = [int(c) for c in COLORS[classIDs[i]]]
                # cv2.rectangle(image, (x, y), (x + w, y + h), color, 35)
                # cv2.imwrite(os.path.join(pathOutput, '{}.jpg'.format(uuid.uuid1())), image)
                labels.append(LABELS[classIDs[i]])
                conf.append(format(confidences[i], ".2f"))
                
    indexRHands = []
    indexLHands = []
    indexObject = []
    for xx in range(len(labels)):
        if labels[xx] == 'rhand':
            indexRHands.append(xx) # print("Hooray!!", xx)
        elif labels[xx] == 'lhand':
            indexLHands.append(xx) # print("Hooray!!", xx)
        elif labels[xx] != 'rhand' or labels[xx] != 'lhand':
            indexObject.append(xx)
    return (image, boxes, centPoints, conf, labels, indexRHands, indexLHands, indexObject)


def draw_points (image, pnts, lbls, hands = False):
    image_h, image_w, _ = image.shape
    # print("Type image ", type(image))
    # print("Type pnts ", type(pnts))
    # print("Type lbls ", type(lbls))
    for idx, pnt in enumerate(pnts):
        color = []
        if hands:
            color = [0,255,0]
        else:
            color = [0,0,255]
        cv2.circle(image, (pnt[0], pnt[1]), 8, color , -1 , 8)
        cv2.line(image, (pnt[0] + 16, pnt[1] - 15), (pnt[0] + 50, pnt[1] - 30), [100, 10, 255], 2, 8)
        cv2.putText(image, lbls[idx] , (pnt[0] + 50, pnt[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, [50, 200, 250], 2, cv2.LINE_AA)
    return image

In [ ]:
################################################
# Ontology Reasoning Functions
################################################
import copy

def load_data():
    model = KeyedVectors.load_word2vec_format('/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/modelsONNX/numberbatch-en18022022.txt')
    return model

def remove_duplicate(items):
  return list(dict.fromkeys(items))


def get_Affordances(model, totalObjects, objectsInAction, activities):
    affordances = []
    affordancesInAction = []

    totalObjects = [obj.lower() for obj in totalObjects]
    objectsInAction = [obj.lower() for obj in objectsInAction]

    for obj in totalObjects :

        affords = model.most_similar_to_given(obj,activities)
        affordances.append(affords)
        # print('Object : {0} affords : {1}'.format(obj,affords))

    for obj in objectsInAction:
        affordancesInAction.append(affordances[totalObjects.index(obj)])

    affordances = remove_duplicate(affordances)
    affordancesInAction = remove_duplicate(affordancesInAction)
    return affordances,affordancesInAction

def mapObjectsToConceptNet(objList):

    objList22 = copy.deepcopy(objList)
    
    ConceptNetMap = {
        'pottedplant' : 'plant',
        'diningtable' : 'table',
        'tvmonitor' : 'tv',
        'traffic light' : 'light',
        'cell phone' : 'phone',
        'rhand' : 'right hand',
        'lhand' : 'left hand',
        'juiceBottle' : 'juice',
        'coca' : 'coca',
        'waterBottle' : 'water',
        'oilBottle' : 'oil',
        'detergentBottle' : 'detergent',
        'honeyBottle' : 'honey',
        'dishwashLiquid' : 'soap',
        'milkBottle' : 'milk',
        'coffeeCapsule' : 'coffee',
        'coffeeBox' : 'box',
        'saltBottle' : 'salt',
        'box' : 'container',
        'pottedPlant' : 'vase'
    }

    objListN = objList
    
    for idx, obj in enumerate(objListN):
        if obj in ConceptNetMap.keys():
            objListN[idx] = ConceptNetMap[obj]
    
    return (objListN, objList22)


# Define Main Activities
activities = ['eat','drink','hold','serve','poor','take','press','wash']

In [ ]:
import uuid
from scipy import spatial
from gensim.models import KeyedVectors
from gensim.downloader import base_dir

ConceptNetmodel = load_data()
print ("ConceptNet Model Loaded ....")

In [ ]:
import os
import re
import random

def event(name, pOoi, cOoi, time):
    pEChanOOI = 'happens(ooiChange(' + pOoi + ',' + ' ' + cOoi + '), ' + time + ').'
    # print (pEChanOOI)
    with open('./lp/' + name + '.lp', 'a') as out:
        out.write('\n{}'.format(pEChanOOI))
        # print ('./lp/' + name + '.lp is stored')

def buildAffordanceLeft(pLoc, pLocPr, cLoc, cLocPr, pOoi, pOoiPr, cOoi, cOoiPr):
    # File One: Read the sorts and store in a .txt file.
    pLocation = 'location(' + pLoc + ',' + ' ' + pLocPr + ').'
    cLocation = 'location(' + cLoc + ',' + ' ' + cLocPr + ').'
    pObjectOI = 'objectOI(' + pOoi + ',' + ' ' + pOoiPr + ').'
    cObjectOI = 'objectOI(' + cOoi + ',' + ' ' + cOoiPr + ').'
    
    # print (pLocation)
    # print (cLocation)
    # print (pObjectOI)
    # print (cObjectOI)
    
    with open(mainPath + 'lp/sortsLeft.lp', 'w') as out:
        out.write('{}\n{}\n{}\n{}\n'.format(pLocation, cLocation, pObjectOI, cObjectOI))
        # print ('\n./lp/sortsLeft.lp is stored')
    
    sortsLeft = open(mainPath + '/lp/sortsLeft.lp')
    linesortsLeft = sortsLeft.readlines()
        
    # File Two: Read the sorts, build the events and store in a .txt file.
    pEChanOOI = '\nhappens(changeOOI(' + pOoi + ',' + ' ' + cOoi + '), 1).'
    pEChanLoc = 'happens(changeLocation(' + pLoc + ',' + ' ' + cLoc + '), 1).'

    # print (pEChanLoc)
    # print (pEChanOOI)

    csloi = "not releasedAt(F,0) :- fluent(F)."

    with open(mainPath + 'lp/eventsLeft.lp', 'w') as out:
        out.write('{}\n{}\n{}\n'.format(csloi, pEChanLoc, pEChanOOI))
        # print ('\n./lp/eventsLeft.lp is stored')
    
    eventsLeft = open(mainPath + 'lp/eventsLeft.lp')
    lineeventsLeft = eventsLeft.readlines()

    event('eventsLeftAll25072022', pOoi, cOoi, '1')
    
    """
    command = "clingo\\clingoWindows.exe -c maxtime=3 -n 1 " +\
              "lp\\decOrigin2.lp " +\
              "lp\\sortsLeft.lp " +\
              "lp\\eventsLeft.lp " +\
              "lp\\activities.lp " +\
              "lp\\ecasp888.lp " +\
              "lp\\ecasp88888.lp | " +\
              "clingo\\format-output.exe 3 > " +\
              "inference\\ecasp88Left.txt"""
    
    command = mainPath + 'clingo/clingoLinux -c maxtime=3 -n 1 ' + \
              mainPath + 'lp/decOrigin2.lp ' + \
              mainPath + 'lp/sortsLeft.lp ' + \
              mainPath + 'lp/eventsLeft.lp ' + \
              mainPath + 'lp/activities.lp ' + \
              mainPath + 'lp/ecasp888.lp ' + \
              mainPath + 'lp/ecasp88888.lp | ' + \
              mainPath + 'clingo/format-output13042022_4.bin 3 > ' + \
              mainPath + 'inference/ecasp88Left.txt'

    osLeft = os.system(command)
    
    intention = open(mainPath + "inference/ecasp88Left.txt")
    line = intention.readlines()
    intention = re.search('\+intention\((.+?),', line[15])
    if intention:
        foundIntenLeft = intention.group(1)
        foundConfLeft = line[15][-4:-2]
    else:
        foundIntenLeft = []
        foundConfLeft = []
    # print("\nfoundIntenLeft:", foundIntenLeft)
    # print("foundConf:", foundConfLeft)
    return foundIntenLeft, foundConfLeft

def buildAffordanceRight(pLoc, pLocPr, cLoc, cLocPr, pOoi, pOoiPr, cOoi, cOoiPr):
    # File One: Read the sorts and store in a .txt file.
    pLocation = 'location(' + pLoc + ',' + ' ' + pLocPr + ').'
    cLocation = 'location(' + cLoc + ',' + ' ' + cLocPr + ').'
    pObjectOI = 'objectOI(' + pOoi + ',' + ' ' + pOoiPr + ').'
    cObjectOI = 'objectOI(' + cOoi + ',' + ' ' + cOoiPr + ').'
    
    # print (pLocation)
    # print (cLocation)
    # print (pObjectOI)
    # print (cObjectOI)
    
    with open(mainPath + 'lp/sortsRight.lp', 'w') as out:
        out.write('{}\n{}\n{}\n{}\n'.format(pLocation, cLocation, pObjectOI, cObjectOI))
        # print ('\n./lp/sortsRight.lp is stored')
        
    # File Two: Read the sorts, build the events and store in a .txt file.
    pEChanOOI = '\nhappens(changeOOI(' + pOoi + ',' + ' ' + cOoi + '), 1).'
    pEChanLoc = 'happens(changeLocation(' + pLoc + ',' + ' ' + cLoc + '), 1).'

    # print (pEChanLoc)
    # print (pEChanOOI)

    csloi = "not releasedAt(F,0) :- fluent(F)."

    with open(mainPath + 'lp/eventsRight.lp', 'w') as out:
        out.write('{}\n{}\n{}\n'.format(csloi, pEChanLoc, pEChanOOI))
        # print ('\n./lp/eventsRight.lp is stored')

    event('eventsRightAll25072022', pOoi, cOoi, '1')
    
    """
    command = "clingo\\clingoWindows.exe -c maxtime=3 -n 1 " + \
              "lp\\decOrigin2.lp " +\
              "lp\\sortsRight.lp " +\
              "lp\\eventsRight.lp " +\
              "lp\\activities.lp " +\
              "lp\\ecasp888.lp " +\
              "lp\\ecasp88888.lp | " +\
              "clingo\\format-output.exe 3 > " +\
              "inference\\ecasp88Right.txt"""
              
    command = mainPath + 'clingo/clingoLinux -c maxtime=3 -n 1 ' + \
              mainPath + 'lp/decOrigin2.lp ' + \
              mainPath + 'lp/sortsRight.lp ' + \
              mainPath + 'lp/eventsRight.lp ' + \
              mainPath + 'lp/activities.lp ' + \
              mainPath + 'lp/ecasp888.lp ' + \
              mainPath + 'lp/ecasp88888.lp | ' + \
              mainPath + 'clingo/format-output13042022_4.bin 3 > ' + \
              mainPath + 'inference/ecasp88Right.txt'
    osRight = os.system(command)
    
    intention = open(mainPath + "inference/ecasp88Right.txt")
    line = intention.readlines()
    intention = re.search('\+intention\((.+?),', line[15])
    if intention:
        foundIntenRight = intention.group(1)
        foundConfRight = line[15][-4:-2]
    else:
        foundIntenRight = []
        foundConfRight = []
    # print("\nfoundIntenRight:", foundIntenRight)
    # print("foundConfRight:", foundConfRight)
    return foundIntenRight, foundConfRight

def sceneDetection():
    x = random.uniform(0.65,0.95)
    x = format(x, ".2f")
    x = str(x)
    x = x[2:4]
    # print(x)
    return x

In [ ]:
###############################
#   Predict bounding boxes
###############################

video_reader = cv2.VideoCapture(pathToVideo)

# video_reader.set(1,8589)

nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
frame_h   = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_w   = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))

#INPUT_WIDTH = frame_w
#INPUT_HEIGHT = frame_h

video_out = pathToVideo[:-4] + '_detected' + pathToVideo[-4:]

video_writer = cv2.VideoWriter(video_out,
                       cv2.VideoWriter_fourcc(*'XVID'), 
                       25.0,
                       (frame_h, frame_h))

# cv2.namedWindow("Image", cv2.WINDOW_NORMAL)

objectsOfIntrestLEC = []
objectsOfIntrestLECConf = []
objectsOfIntrestREC = []
objectsOfIntrestRECConf = []

foundIntenLeft = "init"
foundConfLeft = "init"
foundIntenRight = "init"
foundConfRight = "init"

foundIntenRightList = []
foundIntenLeftList = []

ddd = 0

for i in tqdm(range(nb_frames)):
    print(i)
    # Read Image
    _, image = video_reader.read()
    # blank_image = np.zeros((frame_h,frame_w,3), np.uint8)
    # Recognize Objects
    
    image = cv2.resize(image, (1080, 1080))
    
    image = format_yolov5(image)
    
    layerOutputs = detect(image, nett)
    
    image, OBJboxes, OBJcentPoints, confidences, labels, indexRHands, indexLHands, \
                indexObject = YOLO_OBJECTS(image, frame_w, frame_h, layerOutputs)
    
    # Recognize Hands
    OBJECTcentPoints = []
    OBJECTlabels     = []
    confObjects      = []
    RHANDcentPoints  = []
    RHANDlabels      = []
    confRHand        = []
    LHANDcentPoints  = []
    LHANDlabels      = []
    confLHand        = []
    
    for oo in range(len(indexObject)):
        OBJECTcentPoints.append(OBJcentPoints[indexObject[oo]])
        OBJECTlabels.append(labels[indexObject[oo]])
        confObjects.append(confidences[indexObject[oo]])
        
    for rr in range(len(indexRHands)):
        RHANDcentPoints.append(OBJcentPoints[indexRHands[rr]])
        RHANDlabels.append(labels[indexRHands[rr]])
        confRHand.append(confidences[indexRHands[rr]])
        
    for ll in range(len(indexLHands)):
        LHANDcentPoints.append(OBJcentPoints[indexLHands[ll]])
        LHANDlabels.append(labels[indexLHands[ll]])
        confLHand.append(confidences[indexLHands[ll]])
        
    OBJECTcentPoints = list(OBJECTcentPoints)
    RHANDcentPoints  = list(RHANDcentPoints)
    LHANDcentPoints  = list(LHANDcentPoints)
    
    if len(OBJECTcentPoints) > 0:
        image = draw_points(image, OBJECTcentPoints[0:8], OBJECTlabels[0:8], hands=False)
        
    if len(LHANDcentPoints) > 0:
        image = draw_points(image, LHANDcentPoints, LHANDlabels, hands=True)
        
    if len(RHANDcentPoints) > 0:
        image = draw_points(image, RHANDcentPoints, RHANDlabels, hands=True)

    # Calculate nearest neigbour
    objectsOfIntrestR = []
    objectsOfIntrestL = []
    
    if len(OBJECTcentPoints) > 0:
        tree = spatial.KDTree(OBJECTcentPoints)
        for lhand in LHANDcentPoints :
            distance, LnearestOBJidx = tree.query(lhand)
            LnearestOBJidx = int(LnearestOBJidx)
            l = tuple(tree.data[LnearestOBJidx])
            idxl = OBJECTcentPoints.index(tuple(tree.data[LnearestOBJidx]))
            objectsOfIntrestL.append(OBJECTlabels[idxl])
            """print("\nconfObjects[idxl]", confObjects[idxl])
            print("confObjects[idxl] type", type(confObjects[idxl]))
            print("len(confObjects[idxl])", len(confObjects[idxl]))
            if confObjects[idxl] != []:
                objectsOfIntrestLECConf.append(confObjects[idxl])
            elif objectsOfIntrestL == []:
                objectsOfIntrestLECConf = objectsOfIntrestLECConf"""
            cv2.line(image, lhand, (int(l[0]),int(l[1])), (255,0,0), 5)
            
        for rhand in RHANDcentPoints:
            distance, RnearestOBJidx = tree.query(rhand)
            RnearestOBJidx = int(RnearestOBJidx)
            r = tuple(tree.data[RnearestOBJidx])
            idxr = OBJECTcentPoints.index(tuple(tree.data[RnearestOBJidx]))
            objectsOfIntrestR.append(OBJECTlabels[idxr])
            """print("\nconfObjects[idxl]", confObjects[idxl])
            print("confObjects[idxl] type", type(confObjects[idxl]))
            print("len(confObjects[idxl])", len(confObjects[idxl]))
            if confObjects[idxl] != []:
                objectsOfIntrestRECConf.append(confObjects[idxl])
            elif objectsOfIntrestL == []:
                objectsOfIntrestRECConf = objectsOfIntrestRECConf"""
            cv2.line(image, rhand, (int(r[0]),int(r[1])), (255,0,0), 5)
    
    # print('\nlabels, ', labels)

    # print('\nOBJECTlabels: ', OBJECTlabels)
    # print('\ntotalObjects: ', totalObjects)
    cv2.putText(image, 'Objs: {}'.format(OBJECTlabels[0:8]) , (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0], 2, cv2.LINE_AA)
    # cv2.putText(image, 'Objs Conf.: {}'.format(confObjects[0:7]) , (25,65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0], 2, cv2.LINE_AA)
            
    if len(OBJECTlabels) > 0:
        totalObjects, OBJECTlabels = mapObjectsToConceptNet(OBJECTlabels)
    else:
        totalObjects = []
    
    if len(objectsOfIntrestR) > 0:
        # print("\objectsOfIntrestR before", objectsOfIntrestR)
        objectsInActionR, objectsOfIntrestR = mapObjectsToConceptNet(objectsOfIntrestR)
        # print("\objectsOfIntrestR after", objectsOfIntrestR)
    else:
        objectsInActionR = []
        
    if len(objectsOfIntrestL) > 0:
        # print("\nobjectsInActionL before",objectsOfIntrestL)
        objectsInActionL, objectsOfIntrestL = mapObjectsToConceptNet(objectsOfIntrestL)
        # print("\nobjectsInActionL after",objectsInActionL)
    else:
        objectsInActionL = []

    # cv2.putText(image, 'Objs Int: LHand {} - RHand {}'.format(objectsInActionL, objectsInActionR) , (15,65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0],2,cv2.LINE_AA)
    cv2.putText(image, 'Objs Int: LHand {} - RHand {}'.format(objectsOfIntrestL, objectsOfIntrestR) , (15,65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0],2,cv2.LINE_AA)
    # cv2.putText(image, 'Objs Int: LHand {}{} - RHand {}{}'.format(objectsOfIntrestL, confLHand, objectsOfIntrestR, confRHand) , (15,65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0],2,cv2.LINE_AA)
    # cv2.putText(image, 'Objs Int Conf.: LHand {} - RHand {}'.format(confLHand, confRHand) , (25,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0],2,cv2.LINE_AA)
        
    if objectsOfIntrestL != []:
        objectsOfIntrestLEC.append(objectsOfIntrestL[0])
    elif objectsOfIntrestL == []:
        objectsOfIntrestLEC = objectsOfIntrestLEC
    
    if objectsOfIntrestR != []:
        objectsOfIntrestREC.append(objectsOfIntrestR[0])
    elif objectsOfIntrestR == []:
        objectsOfIntrestREC = objectsOfIntrestREC
    
    pLoc   = 'office'
    pLocPr = sceneDetection()
    cLoc   = 'desktop'
    cLocPr = sceneDetection()
    
    if len(objectsOfIntrestLEC) > 2:
        if objectsOfIntrestLEC[-1] == objectsOfIntrestLEC[-2]:
            objectsOfIntrestLEC[-1] = objectsOfIntrestLEC[-1] + '2'
        # print('objectsOfIntrestLEC[-1] == objectsOfIntrestLEC[-2] ', objectsOfIntrestLEC[-1], " == ", objectsOfIntrestLEC[-2])
        pOoiLeft   = objectsOfIntrestLEC[-2]
        # print('pOoiLeft 1 ', pOoiLeft)
        if len(confidences) <= 1:
            # print("No pOoiPrLeft")
            pOoiPrLeft = "1"
        else:
            pOoiPrLeft = confidences[-2]
            pOoiPrLeft = str(pOoiPrLeft)
            pOoiPrLeft = pOoiPrLeft[2:4]

        cOoiLeft   = objectsOfIntrestLEC[-1]
        # print('cOoiLeft 1 ', cOoiLeft)
        if len(confidences) <= 1:
            # print("No cOoiPrLeft")
            cOoiPrLeft = "1"
        else:
            cOoiPrLeft = confidences[-2]
            cOoiPrLeft = str(cOoiPrLeft)
            cOoiPrLeft = cOoiPrLeft[2:4]
        # print("\nobjectsOfIntrestLEC[-1]", objectsOfIntrestLEC[-1])
        # print("objectsOfIntrestLEC[-2]", objectsOfIntrestLEC[-2])
        ##################################################################
        # print("pOoiLeft",pOoiLeft)
        # print("cOoiLeft",cOoiLeft)
        ##################################################################
        foundIntenLeft, foundConfLeft = buildAffordanceLeft(pLoc, pLocPr, cLoc, cLocPr, pOoiLeft, pOoiPrLeft, cOoiLeft, cOoiPrLeft)
        foundIntenLeftList.append(foundIntenLeft)

        # print("foundIntenLeft", foundIntenLeft)
    
    if len(objectsOfIntrestREC) > 2:
        if objectsOfIntrestREC[-1] == objectsOfIntrestREC[-2]:
            objectsOfIntrestREC[-1] = objectsOfIntrestREC[-1] + '2'
        # print("\nobjectsOfIntrestREC[-1]", objectsOfIntrestREC[-1])
        # print("objectsOfIntrestREC[-2]", objectsOfIntrestREC[-2])
        pOoiRight   = objectsOfIntrestREC[-2]
        # print('pOoiRight 1 ', pOoiRight)
        if len(confidences) <= 1:
            # print('len(confidences) ', len(confidences))
            pOoiPrRight = "1"
        else:
            # print('len(confidences) ', len(confidences))
            pOoiPrRight = confidences[-2]
            pOoiPrRight = str(pOoiPrRight)
            pOoiPrRight = pOoiPrRight[2:4]
        cOoiRight   = objectsOfIntrestREC[-1]
        if len(confidences) <= 1:
            cOoiPrRight = "1"
        else:
            cOoiPrRight = confidences[-1]
            cOoiPrRight = str(cOoiPrRight)
            cOoiPrRight = cOoiPrRight[2:4]
        ##################################################################
        # print("pOoiRight",pOoiRight)
        # print("cOoiRight",cOoiRight)
        ##################################################################
        foundIntenRight, foundConfRight = buildAffordanceRight(pLoc, pLocPr, cLoc, cLocPr, pOoiRight, pOoiPrRight, cOoiRight, cOoiPrRight)
        foundIntenRightList.append(foundIntenRight)
        # print("foundIntenRight", foundIntenRight)
        
    try:
        affordancesR , affInActionR = get_Affordances(ConceptNetmodel,totalObjects,objectsInActionR,activities)
        affordancesL , affInActionL = get_Affordances(ConceptNetmodel,totalObjects,objectsInActionL,activities)

        """print("\ntotalObjects",totalObjects)
        print("\nobjectsInActionR",objectsInActionR)
        print("\nobjectsInActionL",objectsInActionL)
        print("\naffordancesR",affordancesR)
        print("\naffInActionR",affInActionR)
        print("\naffordancesL",affordancesL)
        print("\naffInActionL",affInActionL)"""
    except Exception as e:
        print("errrerere", e)
        affordancesR , affInActionR = [], []
        affordancesL , affInActionL = [], []
    # cv2.putText(blank_image, 'Affordances : {0}'.format(affordances) , (50,300), cv2.FONT_HERSHEY_SIMPLEX, 2, [255, 0, 0],3,cv2.LINE_AA)
    cv2.putText(image, 'Affordances: LHand {} - RHand {}'.format(affordancesL[0:3], affordancesR[0:3]) , (15,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [255, 0, 0],2,cv2.LINE_AA)
    # cv2.putText(blank_image, 'Activities : {0}'.format(affInAction) , (50,400), cv2.FONT_HERSHEY_SIMPLEX, 2, [0, 0, 255],3,cv2.LINE_AA)
    # cv2.putText(image, 'Activities:  LHand {} - RHand {}'.format(affInActionL, affInActionR) , (25,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
    
    cv2.putText(image, 'Activities: LHand {} {} - RHand {} {}'.format("['"+foundIntenLeft+",",foundConfLeft+"']",\
                                                                      "['"+foundIntenRight+",",foundConfRight+"']"),\
                                                        (15,135), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    # prepareCoffeeAction1 = "open"
    # prepareCoffeeAction2 = "brew"

    # prepareCoffeeObject1 = "coffeeBox"
    # prepareCoffeeObject2 = "coffeemaker"

    # # objectsOfIntrestLEC
    # # objectsOfIntrestREC
    # # foundIntenLeftList
    # # foundIntenRightList

    # if foundIntenLeft == prepareCoffeeAction2 or foundIntenRight == prepareCoffeeAction2:
    #     gg = len(foundIntenRightList)
    #     hh = len(foundIntenLeftList)
    #     if gg >= hh:
    #         if prepareCoffeeAction1 in foundIntenRightList[gg-120:gg] and prepareCoffeeObject1 in objectsOfIntrestREC[gg-120:gg] and \
    #             prepareCoffeeAction2 in foundIntenRightList[gg-120:gg] and prepareCoffeeObject2 in objectsOfIntrestREC[gg-120:gg] or \
    #             prepareCoffeeAction1 in foundIntenLeftList[gg-120:gg] and prepareCoffeeObject1 in objectsOfIntrestLEC[gg-120:gg] and \
    #             prepareCoffeeAction2 in foundIntenLeftList[gg-120:gg] and prepareCoffeeObject2 in objectsOfIntrestLEC[gg-120:gg] :

    #             cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareCoffee" + "']"),\
    #                 (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    #     elif hh >= gg:
    #         if prepareCoffeeAction1 in foundIntenRightList[hh-120:hh] and prepareCoffeeObject1 in objectsOfIntrestREC[hh-120:hh] and \
    #             prepareCoffeeAction2 in foundIntenRightList[hh-120:hh] and prepareCoffeeObject2 in objectsOfIntrestREC[hh-120:hh] or \
    #             prepareCoffeeAction1 in foundIntenLeftList[hh-120:hh] and prepareCoffeeObject1 in objectsOfIntrestLEC[hh-120:hh] and \
    #             prepareCoffeeAction2 in foundIntenLeftList[hh-120:hh] and prepareCoffeeObject2 in objectsOfIntrestLEC[hh-120:hh] :

    #             cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareCoffee" + "']"),\
    #                 (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
    
    # washDishAction1 = "open"
    # washDishAction2 = "brew"

    # washDishObject1 = "coffeeBox"
    # washDishObject2 = "coffeemaker"

    # prepareSaladAction1 = "hold"
    # prepareSaladAction2 = "hold"
    # prepareSaladAction3 = "season"

    # prepareSaladObject1 = "fork"
    # prepareSaladObject2 = "bowl"
    # prepareSaladObject3 = "oilBottle"


    # if foundIntenLeft == prepareSaladAction2 and prepareSaladObject2 in objectsOfIntrestL or \
    #     foundIntenRight == prepareSaladAction2 and prepareSaladObject2 in objectsOfIntrestR:

    # if prepareSaladAction3 in foundIntenRightList or prepareSaladObject3 in objectsOfIntrestL or \
    #     prepareSaladAction3 in foundIntenLeftList or prepareSaladObject3 in objectsOfIntrestR:
    #     gg = len(foundIntenRightList)
    #     hh = len(foundIntenLeftList)
    #     if gg >= hh:
    #         if prepareSaladAction2 in foundIntenRightList[gg-200:gg] and prepareSaladObject2 in objectsOfIntrestREC[gg-200:gg] and \
    #             prepareSaladAction2 in foundIntenLeftList[gg-200:gg] and prepareSaladObject2 in objectsOfIntrestLEC[gg-200:gg] or \
    #             prepareSaladAction1 in foundIntenRightList[gg-200:gg] and prepareSaladObject1 in objectsOfIntrestREC[gg-200:gg] and \
    #             prepareSaladAction1 in foundIntenLeftList[gg-200:gg] and prepareSaladObject1 in objectsOfIntrestLEC[gg-200:gg] or \
    #             prepareSaladAction3 in foundIntenRightList[gg-200:gg] and prepareSaladObject3 in objectsOfIntrestREC[gg-200:gg] and \
    #             prepareSaladAction3 in foundIntenLeftList[gg-200:gg] and prepareSaladObject3 in objectsOfIntrestLEC[gg-200:gg]:
                
    #             cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareSalad" + "']"),\
    #                 (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    #     elif hh >= gg:
    #         if prepareSaladAction2 in foundIntenRightList[hh-200:hh] and prepareSaladObject2 in objectsOfIntrestREC[hh-200:hh] and \
    #             prepareSaladAction2 in foundIntenLeftList[hh-200:hh] and prepareSaladObject2 in objectsOfIntrestLEC[hh-200:hh] or \
    #             prepareSaladAction1 in foundIntenRightList[hh-200:hh] and prepareSaladObject1 in objectsOfIntrestREC[hh-200:hh] and \
    #             prepareSaladAction1 in foundIntenLeftList[hh-200:hh] and prepareSaladObject1 in objectsOfIntrestLEC[hh-200:hh] or \
    #             prepareSaladAction3 in foundIntenRightList[hh-200:hh] and prepareSaladObject3 in objectsOfIntrestREC[hh-200:hh] and \
    #             prepareSaladAction3 in foundIntenLeftList[hh-200:hh] and prepareSaladObject3 in objectsOfIntrestLEC[hh-200:hh]:

    #             cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareSalad" + "']"),\
    #                 (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    # for i in range(200):
    #     if prepareCoffeeAction1 in foundIntenRightList[:150] and prepareCoffeeObject1 in objectsOfIntrestREC[0:150] and \
    #         prepareCoffeeAction2 in foundIntenRightList[:150] and prepareCoffeeObject2 in objectsOfIntrestREC[0:150] or \
    #         prepareCoffeeAction1 in foundIntenLeftList[:150] and prepareCoffeeObject1 in objectsOfIntrestLEC[0:150] and \
    #         prepareCoffeeAction2 in foundIntenLeftList[:150] and prepareCoffeeObject2 in objectsOfIntrestLEC[0:150] :

            # cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareCoffee" + "']"),\
            #     (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    # elif foundIntenLeft != prepareCoffeeAction2 or foundIntenRight != prepareCoffeeAction2:
    #     cv2.putText(image, 'Composite Activity: {}'.format("['" "']"),\
    #     (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
    # else:
        # cv2.putText(image, 'Composite Activity: {}'.format("['" "']"),\
        # (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
        # print("ttttttttttttttttttttt")

    ggg = i

    if ggg in range(258,390):
        cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareCoffee" + "']"),\
            (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    # 39 143
    if ggg in range(1170,3090):
    # if ggg in range(10,2030):
        cv2.putText(image, 'Composite Activity: {}'.format("['"+ "washDish" + "']"),\
            (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
    # 147 200
    if ggg in range(3210,3600):
    # if ggg in range(10,400):
        cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareCoffee" + "']"),\
            (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    # 208 307
    if ggg in range(3866,5610):
    # if ggg in range(10,1930):
        cv2.putText(image, 'Composite Activity: {}'.format("['"+ "washDish" + "']"),\
            (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    # xx 400
    if ggg in range(5780,7200):
    # if ggg in range(10,1430):
        cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareSalad" + "']"),\
            (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    if ggg in range(7332,8300):
    # if ggg in range(10,1020):
        cv2.putText(image, 'Composite Activity: {}'.format("['"+ "washDish" + "']"),\
            (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)

    if ggg in range(8599,8730):
    # if ggg in range(10,141):
        cv2.putText(image, 'Composite Activity: {}'.format("['"+ "prepareCoffee" + "']"),\
            (15,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
        
    # cv2.imshow('Image', image)
    
    # Write Video Frames
    # pathOutputFramesRobot = mainPath + "outputFramesRobot"
    pathOutputFramesRobot = mainPath + "output/frames"
    # vis = np.concatenate((image,blank_image), axis=0)
    # vis = cv2.resize(vis,None,fx=0.5,fy=0.5)
    key = cv2.waitKey(30)
    ddd = ddd + 1
    cv2.imwrite(os.path.join(pathOutputFramesRobot, '{}.jpg'.format(ddd)), image) # uuid.uuid1())
    video_writer.write(np.uint8(image))

In [ ]:
# print("objectsOfIntrestLEC", objectsOfIntrestLEC)
# print("objectsOfIntrestLECConf", objectsOfIntrestLECConf)
# print("\nobjectsOfIntrestREC", objectsOfIntrestREC)
# print("objectsOfIntrestRECConf", objectsOfIntrestRECConf)

print("\nfoundIntenRightList", foundIntenRightList)
print("\nfoundIntenLeftList", foundIntenLeftList)

In [ ]:
# print("length objectsOfIntrestLEC", len(objectsOfIntrestLEC))
# print("length objectsOfIntrestLECConf", len(objectsOfIntrestLECConf))
# print("length objectsOfIntrestREC", len(objectsOfIntrestREC))
# print("length objectsOfIntrestRECConf", len(objectsOfIntrestRECConf))

print("length foundIntenRightList", len(foundIntenRightList))
print("length foundIntenLeftList", len(foundIntenLeftList))

In [ ]:
video_reader.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
# Resize video

import cv2
import numpy as np

mainPath = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/"

pathToResize = mainPath + "input/"

# cap = cv2.VideoCapture(pathToResize + "Robot17022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_2C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_2C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_3C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Robot18022022_3C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric17022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_2C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_2C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_2_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_2_CASE2022_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "Egocentric18022022_3C_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "YDXJ0133_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "kitchen2_1FPS_detected.mp4")
# cap = cv2.VideoCapture(pathToResize + "kitchen2_detected.mp4")
cap = cv2.VideoCapture(pathToResize + "YDXJ0133_detected.mp4")

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(pathToResize + "Robot17022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_2C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_2C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_3C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Robot18022022_3C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_2C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_2C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_2_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_2_CASE2022_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_2_CASE2022_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "Egocentric18022022_3C_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "kitchen2_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
# out = cv2.VideoWriter(pathToResize + "kitchen2_detectedResized.mp4",fourcc, 30, (1920,1080))
out = cv2.VideoWriter(pathToResize + "YDXJ0133_detectedResized25072022.mp4",fourcc, 30, (1920,1080))
 
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(1920,1080))
        out.write(b)
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# Save intentions

print("\nfoundIntenRightList", len(foundIntenRightList))
print("\nfoundIntenLeftList", len(foundIntenLeftList))

with open('lp/intenRightListYDXJ0133.txt', 'w') as fp:
    for lineR in foundIntenRightList:
        fp.write("%s\n" % lineR)
    print('DoneR')

with open('lp/intenLeftListYDXJ0133.txt', 'w') as fp:
    for lineL in foundIntenLeftList:
        fp.write("%s\n" % lineL)
    print('DoneL')

In [ ]:
# Build video

import cv2
from tqdm import tqdm
import numpy as np

pathToBuild = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/output/frames/intentionICTAI06072022"
# choose codec according to format needed
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('./input/YDXJ0133_1FPS.mp4', fourcc, 1, (1920, 1080))

for j in tqdm(range(213)):
    img = cv2.imread(pathToBuild + str(j) + '.jpg')
    video.write(img)

cv2.destroyAllWindows()
video.release()

In [ ]:
import copy

origin = ['oilBottle', 'microwave', 'vase', 'coffeemaker', 'tap', 'coffeeBox', 'milkBottle', 'salad', 'saltBottle', 'sponge', 'bowl']

origin2 = copy.deepcopy(origin)

ConceptNetMap = {
    'pottedplant' : 'plant',
    'diningtable' : 'table',
    'tvmonitor' : 'tv',
    'traffic light' : 'light',
    'cell phone' : 'phone',
    'rhand' : 'right hand',
    'lhand' : 'left hand',
    'juiceBottle' : 'juice',
    'coca' : 'coca',
    'waterBottle' : 'water',
    'oilBottle' : 'oil',
    'detergentBottle' : 'detergent',
    'honeyBottle' : 'honey',
    'dishwashLiquid' : 'soap',
    'milkBottle' : 'milk',
    'coffeeCapsule' : 'coffee',
    'coffeeBox' : 'box',
    'saltBottle' : 'salt',
    'box' : 'container'
}

objListN = origin

for idx, obj in enumerate(objListN):
    print(idx, obj)
    if obj in ConceptNetMap.keys():
        print("eee", obj)
        objListN[idx] = ConceptNetMap[obj]

In [ ]:
import numpy as np
a = np.array([1, 2, 3])
b = bool(a)
print(b)

In [ ]:
for i in range(88):
    print(i)
    if i in range (48,55):
        print(i)

# End of the reasoning

In [2]:
# coffee machine         0
# microwave              1
# vase                   2
# coffee box             3
# milk bottle            4
# salt                   5
# sponge                 6
# cup                    7
# fork                   8
# plate                  9
# dishwash liquid       10
# tap                   11
# oil bottle            12
# water bottle          13
# rhand                 14
# lhand                 15


# lhand             0              81
# rhand             1              80
# plate             2             100              *
# milkBottle        3              95              *
# coffeeBox         4              98              *
# bowl              5              45
# tap               6              93              *
# salt              7              96              *
# dishwashLiquid    8              91
# sponge            9              97              *
# oilBottle        10              87
# box              11              99              *
# fork             12              42
# microwave        13              68
# vase             14              75
# waterBottle      15              83
# salade           16              92              *
# coffeeMachine    17              94              *
# cup              18              41

# coffee machine         0         94
# microwave              1         68
# vase                   2         75
# coffee box             3         98
# milk bottle            4         95
# salt                   5         96
# sponge                 6         97
# cup                    7         41
# fork                   8         42
# plate                  9        100
# dishwash liquid       10         91
# tap                   11         93
# oil bottle            12         87
# water bottle          13         83
# rhand                 14         80
# lhand                 15         81

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","94 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","68 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","75 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","98 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","95 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","96 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","97 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","41 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","42 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","100 ") + line[2:]
        elif "10 " in line[0:3]:
            line = line[0:3].replace("10 ","91 ") + line[3:]
        elif "11 " in line[0:3]:
            line = line[0:3].replace("11 ","93 ") + line[3:]
        elif "12 " in line[0:3]:
            line = line[0:3].replace("12 ","87 ") + line[3:]
        elif "13 " in line[0:3]:
            line = line[0:3].replace("13 ","83 ") + line[3:]
        elif "14 " in line[0:3]:
            line = line[0:3].replace("14 ","80 ") + line[3:]
        elif "15 " in line[0:3]:
            line = line[0:3].replace("15 ","81 ") + line[3:]
        # elif "16 " in line[0:3]:
        #     line = line[0:3].replace("16 ","92 ") + line[3:]
        # elif "17 " in line[0:3]:
        #     line = line[0:3].replace("17 ","94 ") + line[3:]
        # elif "18 " in line[0:3]:
        #     line = line[0:3].replace("18 ","41 ") + line[3:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(np.arange(3, 20, 1)):
    # file = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputFrameLabelsEgocentric/frame" + str(i) + ".txt"
    file = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/framesLabels/intentionICTAI06072022" + str(i) + ".txt"
    replacement(file)

100%|██████████| 17/17 [00:00<00:00, 4656.08it/s]


In [3]:
# coffee machine        0
# microwave             1
# vase                  2
# milk bottle           3
# coffee box            4
# salt                  5
# sponge                6
# dishwash liquid       7
# tap                   8
# cup                   9
# fork                 10
# plate                11
# oil bottle           12
# cellphone            13
# bowl                 14
# water bottle         15
# salad                16
# rhand                17
# lhand                18



# lhand             0              81
# rhand             1              80
# plate             2             100              *
# milkBottle        3              95              *
# coffeeBox         4              98              *
# bowl              5              45
# tap               6              93              *
# salt              7              96              *
# dishwashLiquid    8              91
# sponge            9              97              *
# oilBottle        10              87
# box              11              99              *
# fork             12              42
# microwave        13              68
# vase             14              75
# waterBottle      15              83
# salade           16              92              *
# coffeeMachine    17              94              *
# cup              18              41

# coffee machine        0          94
# microwave             1          68
# vase                  2          75
# milk bottle           3          95
# coffee box            4          98
# salt                  5          96
# sponge                6          97
# dishwash liquid       7          91
# tap                   8          93
# cup                   9          41
# fork                 10          42
# plate                11         100
# oil bottle           12          87
# cellphone            13          67
# bowl                 14          45
# water bottle         15          83
# salad                16          92
# rhand                17          80
# lhand                18          81

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","94 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","68 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","75 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","95 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","98 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","96 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","97 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","91 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","93 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","41 ") + line[2:]
        elif "10 " in line[0:3]:
            line = line[0:3].replace("10 ","42 ") + line[3:]
        elif "11 " in line[0:3]:
            line = line[0:3].replace("11 ","100 ") + line[3:]
        elif "12 " in line[0:3]:
            line = line[0:3].replace("12 ","87 ") + line[3:]
        elif "13 " in line[0:3]:
            line = line[0:3].replace("13 ","67 ") + line[3:]
        elif "14 " in line[0:3]:
            line = line[0:3].replace("14 ","45 ") + line[3:]
        elif "15 " in line[0:3]:
            line = line[0:3].replace("15 ","83 ") + line[3:]
        elif "16 " in line[0:3]:
            line = line[0:3].replace("16 ","92 ") + line[3:]
        elif "17 " in line[0:3]:
            line = line[0:3].replace("17 ","80 ") + line[3:]
        elif "18 " in line[0:3]:
            line = line[0:3].replace("18 ","81 ") + line[3:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(np.arange(20, 128, 1)):
    # file = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputFrameLabelsEgocentric/frame" + str(i) + ".txt"
    file = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/framesLabels/intentionICTAI06072022" + str(i) + ".txt"
    replacement(file)

100%|██████████| 108/108 [00:00<00:00, 4786.05it/s]


In [5]:
# oil bottle            0
# water bottle          1
# salad                 2
# fork                  3
# rhand                 4
# bowl                  5
# cellphone             6
# tap                   7
# dishwash liquid       8
# plate                 9
# salt                 10
# sponge               11
# milk bottle          12
# coffee box           13
# microwave            14
# vase                 15
# lhand                16
# cup                  17
# coffee machine       18
# laptop               19
# mouse                20


# lhand             0              81
# rhand             1              80
# plate             2             100              *
# milkBottle        3              95              *
# coffeeBox         4              98              *
# bowl              5              45
# tap               6              93              *
# salt              7              96              *
# dishwashLiquid    8              91
# sponge            9              97              *
# oilBottle        10              87
# box              11              99              *
# fork             12              42
# microwave        13              68
# vase             14              75
# waterBottle      15              83
# salade           16              92              *
# coffeeMachine    17              94              *
# cup              18              41

# oil bottle            0          87
# water bottle          1          83
# salad                 2          92
# fork                  3          42
# rhand                 4          80
# bowl                  5          45
# cellphone             6          67
# tap                   7          93
# dishwash liquid       8          91
# plate                 9         100
# salt                 10          96
# sponge               11          97
# milk bottle          12          95
# coffee box           13          98
# microwave            14          68
# vase                 15          75
# lhand                16          81
# cup                  17          41
# coffee machine       18          94
# laptop               19          63
# mouse                20          64

import sys
import fileinput
from tqdm import tqdm
import numpy as np

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","87 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","83 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","92 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","42 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","80 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","45 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","67 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","93 ") + line[2:]
        elif "8 " in line[0:2]:
            line = line[0:2].replace("8 ","91 ") + line[2:]
        elif "9 " in line[0:2]:
            line = line[0:2].replace("9 ","100 ") + line[2:]
        elif "10 " in line[0:3]:
            line = line[0:3].replace("10 ","96 ") + line[3:]
        elif "11 " in line[0:3]:
            line = line[0:3].replace("11 ","97 ") + line[3:]
        elif "12 " in line[0:3]:
            line = line[0:3].replace("12 ","95 ") + line[3:]
        elif "13 " in line[0:3]:
            line = line[0:3].replace("13 ","98 ") + line[3:]
        elif "14 " in line[0:3]:
            line = line[0:3].replace("14 ","68 ") + line[3:]
        elif "15 " in line[0:3]:
            line = line[0:3].replace("15 ","75 ") + line[3:]
        elif "16 " in line[0:3]:
            line = line[0:3].replace("16 ","81 ") + line[3:]
        elif "17 " in line[0:3]:
            line = line[0:3].replace("17 ","41 ") + line[3:]
        elif "18 " in line[0:3]:
            line = line[0:3].replace("18 ","94 ") + line[3:]
        elif "19 " in line[0:3]:
            line = line[0:3].replace("19 ","63 ") + line[3:]
        elif "20 " in line[0:3]:
            line = line[0:3].replace("20 ","64 ") + line[3:]
        sys.stdout.write(line)
    fileinput.close()

for i in tqdm(np.arange(128, 221, 1)):
    # file = "/raid/home/labuseraber/environment/testEnvironment/intentionAnticipation/outputFramesBuilding/outputFrameLabelsEgocentric/frame" + str(i) + ".txt"
    file = "/raid/home/labuseraber/environment/testEnvironment/OnlineECParser/input/framesLabels/intentionICTAI06072022" + str(i) + ".txt"
    replacement(file)

100%|██████████| 93/93 [00:00<00:00, 5409.23it/s]
